In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn import metrics, model_selection
import statsmodels.formula.api as smf
from sklearn.pipeline import make_pipeline

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#### HDB Resale Price Predictions 

For today's lab, we will continue working with the HDB resale dataset, and start to explore metrics around the accuracy of the linear models we generate https://data.gov.sg/dataset/resale-flat-prices

In [6]:
hdb = pd.read_csv('data/resale-flat-prices-based-on-registration-date-from-march-2012-onwards.csv')

In [7]:
hdb = hdb.rename(columns={'month': 'year-month'})
hdb['year'] = hdb['year-month'].apply(lambda x: int(x.split("-")[0]))
hdb['month'] = hdb['year-month'].apply(lambda x: int(x.split("-")[1]))
hdb['lower_storey_bound'] = hdb['storey_range'].apply(lambda x: int(x.split()[0]))
hdb['upper_storey_bound'] = hdb['storey_range'].apply(lambda x: int(x.split()[2]))
hdb['flat_age'] = hdb['year'] - hdb['lease_commence_date']
town_dummies = pd.get_dummies(hdb.town, drop_first=True, prefix='TOWN')
hdb_town = pd.concat([hdb, town_dummies], axis=1)
factors = np.concatenate((town_dummies.columns, ["upper_storey_bound", "floor_area_sqm", "flat_age"]), axis=0)

#### 1. Using model_selection.train_test_split, divide the hdb_town data into a training and testing dataset with random_state = 1

In [9]:
model_selection.train_test_split?

In [8]:
hdb_train, hdb_test = model_selection.train_test_split(hdb_town, random_state=1)
print hdb_train.shape, hdb_test.shape

(72473, 40) (24158, 40)


#### 2. Create and fit a Linear Regression Model using Sklearn on the training dataset using the factors provided

In [14]:
lm = LinearRegression().fit(hdb_train[factors], hdb_train["resale_price"])

#### 3. Calculate the MAE, MSE, and RMSE for predictions on the training data 

In [16]:
train_preds = lm.predict(hdb_train[factors])

In [17]:
train_mae = metrics.mean_absolute_error(train_preds, hdb_train["resale_price"])
print train_mae

41750.5048962


In [18]:
train_mse = metrics.mean_squared_error(train_preds, hdb_train["resale_price"])
print train_mse

2920561379.39


In [19]:
train_rmse = np.sqrt(train_mse)
print train_rmse

54042.2184906


#### 4. Calculate the MAE, MSE, and RMSE for predictions on the testing data 

In [20]:
test_preds = lm.predict(hdb_test[factors])

In [21]:
test_mae = metrics.mean_absolute_error(test_preds, hdb_test["resale_price"])
print test_mae

41866.6305437


In [22]:
test_mse = metrics.mean_squared_error(test_preds, hdb_test["resale_price"])
print test_mse

2921704935.12


In [23]:
test_rmse = np.sqrt(test_mse)
print test_rmse

54052.7976623


#### 5. Using the entire dataset i.e. hdb_town, perform 10 folds cross validation. Compute the RMSE for each model fold and the overall mean RMSE 

In [24]:
kf = model_selection.KFold(n_splits=10, shuffle=True)

In [25]:
rmse_list = []
for train_index, test_index in kf.split(hdb_town):
    hdb_kf_train = hdb_town.iloc[train_index]
    hdb_kf_test = hdb_town.iloc[test_index]
    lm_kf = LinearRegression().fit(hdb_kf_train[factors], hdb_kf_train["resale_price"])
    hdb_kf_preds = lm_kf.predict(hdb_kf_test[factors])
    rmse = np.sqrt(metrics.mean_squared_error(hdb_kf_preds, hdb_kf_test["resale_price"]))
    rmse_list.append(rmse)
    print "RMSE:", rmse
print "Mean RMSE:", np.mean(rmse_list)

RMSE: 53759.1380852
RMSE: 54150.3268562
RMSE: 54583.6022947
RMSE: 54499.523151
RMSE: 53375.7205116
RMSE: 53723.6953685
RMSE: 53723.5126501
RMSE: 54518.5672144
RMSE: 53897.0368017
RMSE: 54397.2935896
Mean RMSE: 54062.8416523


#### 6. Now split the dataset using year >= 2016 for testing and year < 2016 for training

In [26]:
hdb_test = hdb_town[hdb_town["year"] >= 2016]
hdb_train = hdb_town[hdb_town["year"] < 2016]
print hdb_test.shape, hdb_train.shape

(25395, 40) (71236, 40)


#### 7. Train a linear model on with this new training dataset and compute the RMSE of this model against both the  training and test data 

In [27]:
lm = LinearRegression().fit(hdb_train[factors], hdb_train["resale_price"])
train_preds = lm.predict(hdb_train[factors])
test_preds = lm.predict(hdb_test[factors])

In [28]:
train_rmse = np.sqrt(metrics.mean_squared_error(train_preds, hdb_train["resale_price"]))
test_rmse = np.sqrt(metrics.mean_squared_error(test_preds, hdb_test["resale_price"]))

print train_rmse 
print test_rmse

51990.462625
60871.419329


#### Bonus Question 1

Housing prices are often estimated by psf depending on the location. This might be a better way to predict the price instead of having a coefficient for area and a separate one for location. 

Create an additional set of columns with the prefix AREA\_TOWN\_ that contains the area if the flat is in that town or is 0 otherwise. Include these columns into the model and run a 10 fold cross validation to see if this improves the model.

Note that you should keep the main terms in a model when interaction terms are included.

Hint: Check the multiply function of dataframes

In [29]:
area_dummies = town_dummies.multiply(hdb["floor_area_sqm"], axis="index").add_prefix('AREA_')

In [31]:
hdb_area = pd.concat([hdb_town, area_dummies], axis=1)
new_factors = np.concatenate((town_dummies.columns, area_dummies.columns, ["upper_storey_bound", "floor_area_sqm", "flat_age"]), axis=0)

In [32]:
rmse_list = []
for train_index, test_index in kf.split(hdb_area):
    hdb_kf_train = hdb_area.iloc[train_index]
    hdb_kf_test = hdb_area.iloc[test_index]
    lm_kf = LinearRegression().fit(hdb_kf_train[new_factors], hdb_kf_train["resale_price"])
    hdb_kf_preds = lm_kf.predict(hdb_kf_test[new_factors])
    rmse = np.sqrt(metrics.mean_squared_error(hdb_kf_preds, hdb_kf_test["resale_price"]))
    rmse_list.append(rmse)
    print "RMSE:", rmse
print "Mean RMSE:", np.mean(rmse_list)

RMSE: 50819.0985734
RMSE: 50436.1048783
RMSE: 50198.9722465
RMSE: 49597.9013981
RMSE: 49884.4088342
RMSE: 49549.1801801
RMSE: 50381.9343533
RMSE: 49485.0095515
RMSE: 50656.6117103
RMSE: 49560.3383809
Mean RMSE: 50056.9560106


#### Ridge and Lasso Regression 

In [33]:
hdb_area.shape

(96631, 65)

In [34]:
hdb_random = pd.DataFrame(np.random.randn(hdb_area.shape[0],200))
hdb_full = pd.concat([hdb_area,hdb_random], axis=1)

In [35]:
hdb_full.head()

,year-month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,190,191,192,193,194,195,196,197,198,199
0,2012-03,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,...,-1.958005,0.588355,0.840697,-1.528331,0.297381,0.148198,1.478362,-0.539837,0.977207,-0.275596
1,2012-03,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,...,0.680669,-1.680983,0.555307,0.226783,-0.327755,0.835698,0.053979,1.035317,0.422508,0.523688
2,2012-03,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,...,-0.306634,0.232233,1.052895,0.392184,0.297569,0.781066,-0.048975,-0.130066,0.070187,0.000394
3,2012-03,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,...,0.053804,0.919346,1.311095,-0.024909,1.161662,-0.572276,-0.233500,-1.587026,-0.347166,-0.404238
4,2012-03,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,...,-0.175876,0.308328,-0.544246,0.481570,0.129781,-0.742183,-0.366746,1.501854,-0.323555,1.464528


In [66]:
sample_size = 500
hdb_sample = hdb_full.sample(sample_size, random_state=10)
full_factors = np.concatenate((hdb_random.columns, town_dummies.columns, area_dummies.columns, ["upper_storey_bound", "floor_area_sqm", "flat_age"]), axis=0)
kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=10)

In [67]:
mse_list = []
for train_index, test_index in kf.split(hdb_sample):
    hdb_kf_train = hdb_sample.iloc[train_index]
    hdb_kf_test = hdb_sample.iloc[test_index]
    lm_kf = LinearRegression().fit(hdb_kf_train[full_factors], hdb_kf_train["resale_price"])
    hdb_kf_preds = lm_kf.predict(hdb_kf_test[full_factors])
    mse = metrics.mean_squared_error(hdb_kf_preds, hdb_kf_test["resale_price"])
    mse_list.append(mse)
    print "MSE:", mse
print "Mean RMSE:", np.sqrt(np.mean(mse_list))

MSE: 8302863125.56
MSE: 6761805338.69
MSE: 6178427568.36
MSE: 7550727674.04
MSE: 5062596270.38
MSE: 8700945425.82
MSE: 8936817844.02
MSE: 4680757440.76
MSE: 3933935755.67
MSE: 6149600432.6
Mean RMSE: 81399.3101174


In [68]:
from sklearn.linear_model import Ridge, Lasso

In [69]:
pipe = make_pipeline(StandardScaler(), Ridge())
alphas = np.logspace(-4, 4, 9)
gs = model_selection.GridSearchCV(
    estimator=pipe,
    param_grid={'ridge__alpha': alphas},
    scoring='neg_mean_squared_error',
    cv=kf)

gs.fit(hdb_sample[full_factors], hdb_sample["resale_price"])

GridSearchCV(cv=KFold(n_splits=10, random_state=10, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'ridge__alpha': array([  1.00000e-04,   1.00000e-03,   1.00000e-02,   1.00000e-01,
         1.00000e+00,   1.00000e+01,   1.00000e+02,   1.00000e+03,
         1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [70]:
np.sqrt(-gs.best_score_)

74763.89733654671

In [71]:
pipe = make_pipeline(StandardScaler(), Lasso())
#alphas = np.logspace(-3, 5, 9)
alphas = np.linspace(500, 10000, 20)
gs = model_selection.GridSearchCV(
    estimator=pipe,
    param_grid={'lasso__alpha': alphas},
    scoring='neg_mean_squared_error',
    cv=kf)

gs.fit(hdb_sample[full_factors], hdb_sample["resale_price"])

GridSearchCV(cv=KFold(n_splits=10, random_state=10, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lasso', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'lasso__alpha': array([   500.,   1000.,   1500.,   2000.,   2500.,   3000.,   3500.,
         4000.,   4500.,   5000.,   5500.,   6000.,   6500.,   7000.,
         7500.,   8000.,   8500.,   9000.,   9500.,  10000.])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [72]:
gs.best_params_

{'lasso__alpha': 2000.0}

In [73]:
np.sqrt(-gs.best_score_)

62386.665126222833

In [80]:
np.sqrt(-gs.cv_results_["mean_test_score"])

array([ 66906.09861579,  63656.69571253,  62847.43250662,  62386.66512622,
        62532.4014958 ,  62885.1926858 ,  63391.88939697,  64134.11152345,
        64939.26661471,  65851.972928  ,  66903.3366441 ,  68037.59062431,
        69204.03696289,  70364.29566956,  71445.00295263,  72415.02478012,
        73364.90691101,  74265.74248808,  75125.26775101,  75944.17601989])